In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("MySparkApp").master("local[*]").getOrCreate()
from pyspark.sql.functions import col,count,avg, month, year, to_date, current_date, when
spark

In [ ]:
df = spark.read.csv("final_data.csv", header = True,inferSchema = True)

In [ ]:
df.show(5)

+---------------------+----------------+--------------------+--------------+------+-----------+------+------+--------------------+------------+-----+-----+-------+------------------+--------+--------------------+----------+--------------------+----------+------------------+-----------+--------+
|trans_date_trans_time|          cc_num|            merchant|      category|   amt|      first|  last|gender|              street|        city|state|  zip|    lat|              long|city_pop|                 job|       dob|           trans_num| unix_time|         merch_lat| merch_long|is_fraud|
+---------------------+----------------+--------------------+--------------+------+-----------+------+------+--------------------+------------+-----+-----+-------+------------------+--------+--------------------+----------+--------------------+----------+------------------+-----------+--------+
|  2019-08-11 19:38:33|4681601008538160|fraud_Hermann and...|  shopping_pos| 50.81|      Tammy| Davis|     F|776

In [ ]:
df.select("gender").show(5)

+------+
|gender|
+------+
|     F|
|     F|
|     M|
|     M|
|     F|
+------+
only showing top 5 rows



In [ ]:
df.printSchema()

root
 |-- trans_date_trans_time: timestamp (nullable = true)
 |-- cc_num: long (nullable = true)
 |-- merchant: string (nullable = true)
 |-- category: string (nullable = true)
 |-- amt: double (nullable = true)
 |-- first: string (nullable = true)
 |-- last: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- street: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- zip: integer (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- city_pop: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- dob: date (nullable = true)
 |-- trans_num: string (nullable = true)
 |-- unix_time: integer (nullable = true)
 |-- merch_lat: double (nullable = true)
 |-- merch_long: double (nullable = true)
 |-- is_fraud: integer (nullable = true)



## 1. Top 5 cities Where most fraudulent transactions occur

In [ ]:
fraud_df = df.filter(col("is_fraud") == 1)

In [ ]:
fraud_counts = fraud_df.groupBy("city").count()
top_5 = fraud_counts.orderBy(col("count").desc())
top_5.show(5)

+-------------+-----+
|         city|count|
+-------------+-----+
|       Dallas|   39|
|      Houston|   39|
|   Birmingham|   36|
|New York City|   35|
|    Allentown|   34|
+-------------+-----+
only showing top 5 rows



## Most fraudent transaction categories assosciated with fraudulent transactions

In [ ]:
fraud_cat = fraud_df.groupBy("category").count()
top_fraudc = fraud_cat.orderBy(col("count").desc())
top_fraudc.show(1)

+-----------+-----+
|   category|count|
+-----------+-----+
|grocery_pos| 2228|
+-----------+-----+
only showing top 1 row



## 3. top 10 most frequent merchants for fraudelent transactions

In [ ]:
fraud_merchant = fraud_df.groupBy("merchant").count()
top_merch = fraud_merchant.orderBy(col("count").desc())
top_merch.show(10)

+--------------------+-----+
|            merchant|count|
+--------------------+-----+
|   fraud_Kilback LLC|   62|
|  fraud_Rau and Sons|   60|
|   fraud_Kozey-Boehm|   60|
|     fraud_Doyle Ltd|   57|
|    fraud_Terry-Huel|   56|
|      fraud_Kuhn LLC|   55|
|     fraud_Boyer PLC|   55|
|     fraud_Kuhic LLC|   53|
|fraud_Moen, Reing...|   53|
|fraud_Kiehn-Emmerich|   53|
+--------------------+-----+
only showing top 10 rows



## 4. Avg amount spent by each gender

In [ ]:
# Group by gender and calculate the average transaction amount
avg_amtgender = df.groupBy("gender").agg(avg(col("amt")).alias("avg_amount_spent"))

# Show the results
avg_amtgender.show()

+------+------------------+
|gender|  avg_amount_spent|
+------+------------------+
|     F|271.71298766157497|
|     M|330.50721562156156|
+------+------------------+



## 5. top 5 merchants with the highest average transaction amount

In [ ]:
avg_mercamt = df.groupBy("merchant").agg(avg(col("amt")).alias("avg_transaction_amount"))
top_merc = avg_mercamt.orderBy(col("avg_transaction_amount").desc())
top_merc.show(5)

+--------------------+----------------------+
|            merchant|avg_transaction_amount|
+--------------------+----------------------+
|fraud_Bashirian G...|     891.0655102040814|
|     fraud_Kuhic LLC|     869.5223437500001|
|fraud_Schmidt and...|     852.2311111111113|
|fraud_Heathcote, ...|     838.5803225806453|
|   fraud_Kozey-Boehm|      832.279864864865|
+--------------------+----------------------+
only showing top 5 rows



## 6. Age of each customer

In [ ]:
df = df.withColumn("dob", to_date(col("dob"), "yyyy-MM-dd HH:mm:ss"))
df = df.withColumn("age", year(current_date()) - year(col("dob")))
df.select("first", "last", "dob", "age").show()

+-----------+---------+----------+---+
|      first|     last|       dob|age|
+-----------+---------+----------+---+
|      Tammy|    Davis|1977-08-12| 48|
|     Monica|   Tucker|1999-06-06| 26|
|      Scott|   Martin|1967-08-02| 58|
|Christopher|   Grimes|1948-04-11| 77|
|   Mckenzie|   Brooks|1961-12-14| 64|
|       John|   Peters|1979-09-03| 46|
|     Morgan|    Smith|1973-11-14| 52|
|      Allen|     Bell|1974-02-15| 51|
|    Crystal|   Gamble|1985-01-01| 40|
|    Douglas|   Willis|1958-09-10| 67|
|       Mary|Rodriguez|1965-09-27| 60|
|   Margaret|      Lam|1972-10-04| 53|
|  Christian|    Roman|1974-10-15| 51|
|      Jesse|  Roberts|1988-04-15| 37|
|     Steven|    Yoder|1992-06-19| 33|
|   Danielle|    Evans|1991-10-13| 34|
|      Casey|   Howell|1966-11-10| 59|
|   Kristina|  Stewart|1971-04-25| 54|
|   Jennifer|    Black|1981-08-29| 44|
|    Dorothy|   Thomas|1996-11-12| 29|
+-----------+---------+----------+---+
only showing top 20 rows



## 7. top 5 states with the higest number of transcartions

In [ ]:
state_transaction_counts = df.groupBy("state").count()
top_states = state_transaction_counts.orderBy(col("count").desc())
top_states.show(5)

+-----+-----+
|state|count|
+-----+-----+
|   NY| 1290|
|   TX| 1287|
|   PA| 1153|
|   CA|  857|
|   OH|  687|
+-----+-----+



## 8. distribution of transcations over the course of month

In [ ]:
monthly_transaction_counts = df.groupBy(month(col("trans_date_trans_time")).alias("month")).count()
monthly_transaction_counts.orderBy(col("month")).show()

+-----+-----+
|month|count|
+-----+-----+
|    1| 1413|
|    2| 1360|
|    3| 1664|
|    4| 1416|
|    5| 1674|
|    6| 1741|
|    7| 1563|
|    8| 1694|
|    9| 1513|
|   10| 1547|
|   11| 1404|
|   12| 2313|
+-----+-----+



## 10. Spending Pattern based on Age

In [ ]:
df = df.withColumn("age_group",
    when(col("age").between(18, 25), "18-25")
    .when(col("age").between(26, 35), "26-35")
    .when(col("age").between(36, 45), "36-45")
    .when(col("age").between(46, 55), "46-55")
    .when(col("age").between(56, 65), "56-65")
    .when(col("age") > 65, "65+"))

avg_spent= df.groupBy("age_group").agg(avg(col("amt")).alias("avg_amount_spent"))
avg_spent.orderBy("age_group").show()

+---------+------------------+
|age_group|  avg_amount_spent|
+---------+------------------+
|    18-25| 341.1335061728393|
|    26-35|  300.274189364462|
|    36-45| 256.7721430363867|
|    46-55|244.01207727044644|
|    56-65|354.12696461824964|
|      65+| 341.5525411255414|
+---------+------------------+

